#### Imports

In [143]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from skopt import BayesSearchCV
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
import numpy as np
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.metrics import MeanSquaredError
from keras.regularizers import l2
from keras.layers import BatchNormalization
from skopt.space import Categorical
from sklearn.metrics import mean_squared_error
import json

In [144]:
final_dataset = pd.read_csv('../../Datasets/final_dataset.csv', sep='\t')

In [145]:
final_dataset.head()

,periodo,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,stock_final,close_quarter,age
0,201701,20001,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,0
1,201702,20001,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,1
2,201703,20001,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,1,2
3,201704,20001,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,3
4,201705,20001,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,NaN,0,4


In [146]:
columns = ['plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'close_quarter', 'cat1', 'cat2', 'cat3', 'close_quarter', 'y']
non_scalable_columns = ['cat1', 'cat2', 'cat3', 'plan_precios_cuidados', 'close_quarter']
n_features = len(columns)

In [147]:
categorical_columns = ['cat1', 'cat2', 'cat3']

# Transformar las columnas categóricas en numéricas
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    final_dataset[col] = le.fit_transform(final_dataset[col])
    label_encoders[col] = le

In [148]:
grouped = final_dataset.groupby(['product_id', 'cat1', 'cat2', 'cat3'])
display(grouped)

In [149]:
scalers = {col: {} for col in columns}
scaled_data_list = []

for (product_id, _, _, _ ), group in grouped:
    scaled_group = group.copy()
    for col in columns:
        if col not in non_scalable_columns:
            scaler = StandardScaler()
            scaled_group[col] = scaler.fit_transform(group[[col]])
            scalers[col][product_id] = scaler
    scaled_data_list.append(scaled_group)

# Combinar todos los datos escalados en un solo DataFrame
scaled_data = pd.concat(scaled_data_list)

In [150]:
display(scaled_data[columns])

,plan_precios_cuidados,cust_request_qty,cust_request_tn,close_quarter,cat1,cat2,cat3,close_quarter,y
0,0,0.461891,-1.704215,0,1,10,47,0,-1.576908
1,0,-0.046911,-2.039835,0,1,10,47,0,-2.042103
2,0,0.786658,-0.435961,1,1,10,47,1,-0.323111
3,0,-1.703222,-1.074260,0,1,10,47,0,-1.117043
4,0,2.865166,0.273781,0,1,10,47,0,0.353285
...,...,...,...,...,...,...,...,...,...
21615,0,-0.652507,-0.792584,0,2,6,18,0,-0.792584
21616,0,-0.268679,-0.649760,1,2,6,18,1,-0.649760
21617,0,0.115148,-0.595870,0,2,6,18,0,-0.595870
21618,0,-0.012794,-0.290890,0,2,6,18,0,-0.290890


In [151]:
# scaled_data[scaled_data['cat1'] != 0]

#### Funcion para preparar los datos y crear el modelo

El objetivo es predecir 2 dias en el futuro, por lo que la idea es re-armar el dataset. Donde el valor de X sera el conjunto de datos hasta N-2 e Y va a ser N (siendo N la cantidad de ventas para ese producto)

In [152]:
def prepare_data(data,validation=False):
    X, y = [], []
    unique_product_ids = data['product_id'].unique()
    
    max_seq_length = 0
    
    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        max_seq_length = max(max_seq_length, len(product_values))
    
    for product_id in unique_product_ids:
        product_data = data[data['product_id'] == product_id].copy()
        product_values = product_data[columns].values
        
        pad_length = max_seq_length - len(product_values)
        
        # Aplicar padding con ceros a la izquierda para igualar la longitud
        padded_product_values = np.pad(product_values[:, :-1], ((pad_length, 0), (0, 0)), mode='constant')
        
        if validation:
            X.append(padded_product_values)
            y.append(product_id)
        else:
            # Todas las observaciones menos las últimas dos. Como despues necesito agregar esos 2 registros
            # para predecir el future, le agrego un pad left extra de 2 para no romper el shape de la red
            X.append(np.pad(padded_product_values, ((2, 0), (0, 0)), mode='constant')[:-2])
            y.append(product_values[-1, -1])      # Última observación
    
    if validation:
        return np.array(X), np.array(y), max_seq_length
    else:
        return np.array(X), np.array(y), max_seq_length


Definimos la funcion para crear el modelo LSTM, sobre este se ejecutara la optimizacion bayesiana

In [153]:
n_steps = 2  # número de pasos de tiempo
epochs = 100
batch_size = 32
predictions = []

In [154]:
data = scaled_data[columns].values
X, y, max_seq_length = prepare_data(scaled_data)

#### Revisamos que la funcion genere registros coherentes

In [155]:
# Muestro todos los registros del ultimo producto
unique_product_ids = scaled_data['product_id'].unique()
product_data = scaled_data[scaled_data['product_id'] == unique_product_ids[0]] #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft
product_values = product_data[columns].values
display(product_values)

array([[ 0.00000000e+00,  4.61890740e-01, -1.70421544e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         4.70000000e+01,  0.00000000e+00, -1.57690786e+00],
       [ 0.00000000e+00, -4.69107783e-02, -2.03983510e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         4.70000000e+01,  0.00000000e+00, -2.04210334e+00],
       [ 0.00000000e+00,  7.86657667e-01, -4.35961139e-01,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         4.70000000e+01,  1.00000000e+00, -3.23110762e-01],
       [ 0.00000000e+00, -1.70322210e+00, -1.07425999e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         4.70000000e+01,  0.00000000e+00, -1.11704256e+00],
       [ 0.00000000e+00,  2.86516600e+00,  2.73781431e-01,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         4.70000000e+01,  0.00000000e+00,  3.53284671e-01],
       [ 0.00000000e+00,  1.44701708e+00,  3.54905442e-01,
         1.00000000e+00,  1.00000000e+00,  1.000000

In [156]:
display(X[0]) #<= cambiar el indice entre 0 y -1 para ver la diferencia agregada por el padleft

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  4.61890740e-01, -1.70421544e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         4.70000000e+01,  0.00000000e+00],
       [ 0.00000000e+00, -4.69107783e-02, -2.03983510e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         4.70000000e+01,  0.00000000e+00],
       [ 0.00000000e+00,  7.86657667e-01, -4.35961139e-01,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         4.70000000e+01,  1.00000000e+00],
       [ 0.00000000e+00, -1.70322210e+00, -1.07425999e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+01,
         4.70000000e+01,  0.00000000e+00],
       [ 0.00000000e+00,  2.865166

In [157]:
display(y[0]) #<= cambiar el indice entre 0 y -1

0.3617453793178112

In [158]:
display(len(X[0]))

36

#### Entrenamiento del modelo

In [159]:
display(max_seq_length)

36

In [160]:
from keras.regularizers import l2

def create_model(units, learning_rate, optimizer='adam', dropout=0.1, activation="tanh", depth=2, l2_penalty=1e-6):
    display(f"Creating model with parameters: units={units}, learning_rate={learning_rate}, "
                f"optimizer={optimizer}, dropout={dropout}, activation={activation}, "
                f"depth={depth}, l2_penalty={l2_penalty}")
    
    model = Sequential()
    
    for i in range(depth):
        if i == 0:
            model.add(LSTM(units=units, return_sequences=True, input_shape=(max_seq_length, n_features - 1), activation=activation, kernel_regularizer=l2(l2_penalty)))
            model.add(BatchNormalization())
        else:
            model.add(LSTM(units=units, return_sequences=True, activation=activation, kernel_regularizer=l2(l2_penalty)))
            model.add(Dropout(dropout))
            model.add(BatchNormalization())
    
    model.add(LSTM(units=units, return_sequences=False, activation=activation, kernel_regularizer=l2(l2_penalty)))
    model.add(Dropout(dropout))
    
    model.add(Dense(1))
    
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    
    # Usamos el error cuadratico medio, porque penaliza errores grandes. Como tenemos ciertos
    # productos "estrella", para los cuales hay mas ventas, y tener un error en lo mismos
    # puede implicar una gran diferencia de las estimaciones, decidimos probar con esta funcion de perdida.
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[MeanSquaredError()])
    return model

#### Armado de los modelos

In [161]:
param_space = {
    'depth': [1, 2, 3, 4],#, 5, 6],
    'activation': ['relu', 'tanh'], #'selu', 'swish'
    'units': [64, 512],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': Categorical(categories=(0.0001, 0.001), prior=None),#0.01
    'epochs': [10, 20, 30, 50, 100],
    'batch_size': Categorical([32, 128], prior=None),#[32, 64, 128],
    'optimizer': ['adam', 'rmsprop', 'sgd'],
    'l2_penalty': [1e-6, 1e-1]
}

In [162]:
# early_stopping = EarlyStopping(monitor='val_loss', patience=5) mean_squared_error
early_stopping = EarlyStopping(monitor='mean_squared_error', patience=5)
callbacks = [early_stopping]

model = KerasRegressor(
    build_fn=create_model,
    verbose=1,
    units=64,
    learning_rate=0.01,
    dropout=0.1,
    activation="tanh",
    depth=1,
    l2_penalty=1e-6,
    callbacks = callbacks
)

# Como ahora cada observacion, representa la serie de un producto, no hace falta usar 
# un cv especifico de time series, con el cv normal deberia ser suficiente.
# cv = TimeSeriesSplit(n_splits=5).split(X)

bayes_search = BayesSearchCV(
    model,
    param_space,
    # n_iter=50,
    cv=10,
    verbose=1,
    # n_jobs=-1,
    scoring='neg_mean_squared_error',
)

/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/skopt/space/space.py:116: UserWarning: Dimension [64, 512] was inferred to Integer(low=64, high=512, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(64, 512), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/Users/mmarchetta/Desktop/LaboratorioIII/my_env/lib/python3.10/site-packages/skopt/space/space.py:116: UserWarning: Dimension [1e-06, 0.1] was inferred to Real(low=1e-06, high=0.1, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(1e-06, 0.1), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(


In [163]:
bayes_search.fit(X, y, verbose=1)

#### Guardo los mejores hiperparametros

In [ ]:
best_params = bayes_search.cv_results_['params'][bayes_search.best_index_]
display(best_params)

best_model = create_model(
    units=best_params['units'],
    learning_rate=best_params['learning_rate'],
    optimizer=best_params['optimizer'],
    dropout=best_params['dropout'],
    activation=best_params['activation'],
    depth=best_params['depth'],
    l2_penalty=best_params['l2_penalty'],
)
best_model.fit(X, y, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)

display(best_params)
display(best_model)

In [ ]:
y_pred = best_model.predict(X)
mean_squared_error(y, y_pred)

In [ ]:
# Obtener los mejores hiperparámetros
best_params = bayes_search.cv_results_['params'][bayes_search.best_index_]

# Guardar los mejores hiperparámetros en un archivo JSON
with open('best_params.json', 'w') as file:
    json.dump(best_params, file)

print('Los mejores hiperparámetros han sido guardados en best_params.json.')


In [ ]:
with open('best_params.json', 'r') as file:
    best_params = json.load(file)

# Reconstruir el modelo con los mejores hiperparámetros
mejor_modelo = create_model(units=best_params['units'], learning_rate=best_params['learning_rate'], optimizer=best_params['optimizer'], dropout=best_params['dropout'], activation=best_params['activation'], depth=best_params['depth'], l2_penalty=best_params['l2_penalty'])
mejor_modelo.fit(X, y, epochs=best_params['epochs'], batch_size=best_params['batch_size'], verbose=1)
# Verificar la configuración del nuevo modelo
print("Configuración del nuevo modelo:")
print(mejor_modelo.summary())

#### Hago las predicciones

In [ ]:
# best_params = BayesSearchCV.best_params_

# best_units = best_params['build_fn__units']
# best_dropout_rate = best_params['build_fn__dropout_rate']
# best_optimizer = best_params['build_fn__optimizer']

# best_model = create_model(units=best_units, dropout_rate=best_dropout_rate, optimizer=best_optimizer)
# best_model.fit(X, y)

In [ ]:
X_f, product_ids, _ = prepare_data(scaled_data, validation=True)

In [ ]:
scaled_predictions = mejor_modelo.predict(X_f)
display(scaled_predictions)

In [ ]:
scaled_predictions_1d =  scaled_predictions.reshape(-1)
display(scaled_predictions_1d)

In [ ]:
predictions_df = pd.DataFrame({
    'product_id': product_ids,
    'predicted_y': scaled_predictions_1d
})

# Iterar sobre cada fila del DataFrame predictions_df
for index, row in predictions_df.iterrows():
    # Obtener el ID del producto y la predicción escalada para este producto
    product_id = row['product_id']
    scaled_prediction = row['predicted_y']
    
    # Obtener el escalador correspondiente a 'predicted_y' para este producto
    scaler = scalers['y'][(product_id)]
    
    # Aplicar la inversa del escalador a la predicción 'predicted_y' para este producto
    inverse_scaled_prediction = scaler.inverse_transform([[scaled_prediction]])[0][0]
    
    # Reemplazar la predicción escalada con la predicción invertida en el DataFrame final
    predictions_df.at[index, 'predicted_y'] = inverse_scaled_prediction

# Guardar el DataFrame final con las predicciones invertidas
predictions_df.to_csv('predictions.csv', index=False)

print('Todas las predicciones han sido generadas y guardadas en predictions.csv después de aplicar la inversa de los scalers.')
